<a href="https://colab.research.google.com/github/aglucky/Basic_Sim/blob/main/Gpt_Neo_Training_Feynman.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports

In [37]:
#Clone the transformers repo into the notebook
!git clone https://github.com/huggingface/transformers

fatal: destination path 'transformers' already exists and is not an empty directory.


In [38]:
import os
os.chdir("transformers")
os.chdir("./examples/language-modeling")
!ls

README.md	  run_clm.py	   run_mlm_no_trainer.py  run_plm.py
requirements.txt  run_mlm_flax.py  run_mlm.py


In [39]:
!pip install -r requirements.txt

In [40]:
!pip install pyarrow --upgrade

Requirement already up-to-date: pyarrow in /usr/local/lib/python3.7/dist-packages (3.0.0)


In [41]:
import os
os.chdir("/content/transformers/examples/")
os.chdir("./language-modeling")

In [42]:
# Need to install latest transformer packages from github so the scripts will run correctly
! pip install git+git://github.com/huggingface/transformers/

  Cloning git://github.com/huggingface/transformers/ to /tmp/pip-req-build-d3iuzwgo
  Running command git clone -q git://github.com/huggingface/transformers/ /tmp/pip-req-build-d3iuzwgo
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for transformers: filename=transformers-4.5.0.dev0-cp37-none-any.whl size=2064484 sha256=c0267e9eda413f5a46c8e223f92960549df53ea5889b052805289f64e744266b
  Stored in directory: /tmp/pip-ephem-wheel-cache-jwrw8fgy/wheels/dc/e5/1e/3a2977a646558fca07585cadcf56aa4a910e995ba945961c4e
Successfully built transformers


#Data Processing

In [43]:
from sklearn.model_selection import train_test_split

with open('/content/Lectures_1.csv', 'r') as data:
  dataset = ["<|title|>" + x.strip() for x in data.readlines()]

  
train, eval = train_test_split(dataset, train_size=.9, random_state=2020)
print("training size:" + str(len(train)))
print("Evaluation size: " + str(len(eval)))

with open('/content/train_tmp.txt', 'w') as file_handle:
  file_handle.write("<|endoftext|>".join(train))
  file_handle.close()

with open('/content/eval_tmp.txt', 'w') as file_handle:
  file_handle.write("<|endoftext|>".join(eval))

training size:80505
Evaluation size: 8946


#Train

In [44]:
!python run_clm.py \
--model_type EleutherAI \
--model_name_or_path EleutherAI/gpt-neo-1.3B \
--train_file "/content/train_tmp.txt" \
--do_train \
--validation_file "/content/eval_tmp.txt" \
--do_eval \
--per_gpu_train_batch_size 1 \
--save_steps -1 \
--block_size 256 \
--num_train_epochs 5 \
--fp16 \
--output_dir="/content/test_model"

2021-04-03 23:52:56.007898: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
04/03/2021 23:52:57 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: True
04/03/2021 23:52:57 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir=/content/test_model, overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=False, evaluation_strategy=IntervalStrategy.NO, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=5.0, max_steps=-1, lr_scheduler_type=SchedulerType.LINEAR, warmup_ratio=0.0, warmup_steps=0, logging_dir=runs/Apr03_23-52-57_e529a484f387, logging_strategy=IntervalStrategy.STEPS, logging_first_ste

In [45]:
# setup imports to use the model
from transformers import GPTNeoForCausalLM, GPT2Tokenizer

model = GPTNeoForCausalLM.from_pretrained("/content/test_model", from_pt=True)
tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")

file /content/test_model/config.json not found


OSError: ignored

#Generate Test

In [ ]:
input_ids = tokenizer.encode("physics", return_tensors='tf')

In [ ]:
# the tf tensor object
input_ids[0]

In [ ]:
generated_text_samples = model.generate(
    input_ids, 
    max_length=150,  
    num_return_sequences=5,
    no_repeat_ngram_size=2,
    repetition_penalty=1.5,
    top_p=0.92,
    temperature=.85,
    do_sample=True,
    top_k=125,
    early_stopping=True
)

In [ ]:
#Print output for each sequence generated above
for i, beam in enumerate(generated_text_samples):
  print("{}: {}".format(i,tokenizer.decode(beam, skip_special_tokens=True)))
  print()